## Event Sourcing with Kinesis

The following demonstration uses Amazon Kinesis Data Streams to trigger AWS Lambda function. As you create the Kinesis trigger, you can configure how the function handles the stream. The Batch size setting determines largest number of records that will be read from your stream at once. The Batch window setting configures the maximum amount of time to gather records before invoking the function.

**Put Item 1**

By configuring the function with a large batch size and a long batch window, you will reduce the cost of the function and increase the duration.
Batch size: 100
Batch window: 30 seconds

**Put Item 2**

By setting a smaller batch size and a shorter batch window you will reduce the duration and increase the cost.
Batch size: 10
Batch window: 2 seconds


![architecture](../images/architecture_1.png "architecture")

**Jupyter Notebook**

Replace ... with the gateway id and execute the script.

```gwid_pattern = …```


In [ ]:
#Set gateway id
gwid_pattern = '...'

import boto3, requests, datetime
from random import randrange

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":3.5},{"size":"Medium","price":4.0},{"size":"Large","price":4.5},{"size":"x-Large","price":5.0}]

url = (f'https://{gwid_pattern}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

for i in range(1000): #Range counter 1000+ will produce more metrics
    accountid = 'a0'
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value-----------
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,           
            'details':{
                'coffeetype': coffeetype_list[randrange(4)],
                'coffeesize': coffeesize_list[coffeesize]["size"],
                'unitprice': coffeesize_list[coffeesize]["price"],
                'quantity': quantity
            }
    }})

print(response)

Lambda Insights shows us that the **Cost** of the **Put Item 2** function, which uses a small batch size, is higher than the **Put Item 1** function which has a larger batch size.

![cost](../images/cost.png "cost")

Lambda Insights shows us that the **Duration** of the **Put Item 1** function, which uses a larger batch size, is higher than the **Put Item 2** function which has a smaller batch size.

![duration](../images/duration.png "duration")

As you might expect, Lambda Insights shows us that the **Network Usage** metric is higher for the **Put Item 2** function, which has a smaller batch size.

![network](../images/network.png "network")